In [ ]:
import numpy as np
import pandas as pd
#import keras
scores = []

In [ ]:
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import h5py

from matplotlib import pyplot as plt
from PIL import Image as Pim

from input_utils import showImg, get_images, crop, expand_rect

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest')
img = load_img('./data/train/white/1-(3.4).png')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
print(x.shape)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='data/preview', save_prefix='cat', save_format='jpeg'):
    print(batch.shape)
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [ ]:
import h5py
h5py.run_tests()

In [ ]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rotation_range=5,
        width_shift_range=0.05,
        height_shift_range=0.05,
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(151, 151),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/test',
        target_size=(151, 151),
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(151, 151, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3 , activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch= 350 // batch_size,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=150 // batch_size)
model.save_weights('models/first_try.h5')  # always save your weights after training or during training

In [ ]:
########## 
# model architecture

#model = Sequential()
#
#model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(151,151,3)))
#model.add(Conv2D(32, (5, 5), activation='relu'))
#
#model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.25))
#
#model.add(Conv2D(16, (3, 3), activation='relu'))
#model.add(Conv2D(10, (3, 3), activation='relu'))
#
#model.add(MaxPooling2D(pool_size=(2,2)))
##model.add(Dropout(0.25))
#
#model.add(Flatten())
#model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(3, activation='softmax'))



In [ ]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy', 
             metrics=['accuracy'])


In [ ]:
#early_stopping = EarlyStopping(monitor='val_loss', patience=2)


In [ ]:
model.fit(X_train, Y_train, batch_size=32, validation_split=.1, 
          epochs=2, verbose=1)

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
scores.append(score)
print(score)

#datagen = ImageDataGenerator(
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest')

Train on 47500 samples, validate on 2500 samples
Epoch 1/10
47500/47500 [==============================] - 17s - loss: 1.8500 - acc: 0.2935 - val_loss: 1.5302 - val_acc: 0.4248
Epoch 2/10
47500/47500 [==============================] - 16s - loss: 1.4836 - acc: 0.4611 - val_loss: 1.2951 - val_acc: 0.5436
Epoch 3/10
47500/47500 [==============================] - 16s - loss: 1.3042 - acc: 0.5345 - val_loss: 1.1538 - val_acc: 0.5804
Epoch 4/10
47500/47500 [==============================] - 16s - loss: 1.1980 - acc: 0.5754 - val_loss: 1.0844 - val_acc: 0.6224
Epoch 5/10
47500/47500 [==============================] - 16s - loss: 1.1024 - acc: 0.6129 - val_loss: 0.9991 - val_acc: 0.6492
Epoch 6/10
47500/47500 [==============================] - 16s - loss: 1.0362 - acc: 0.6367 - val_loss: 1.0078 - val_acc: 0.6436
Epoch 7/10
47500/47500 [==============================] - 16s - loss: 0.9887 - acc: 0.6578 - val_loss: 0.9196 - val_acc: 0.6700
Epoch 8/10
47500/47500 [==============================] - 16s - loss: 0.9528 - acc: 0.6706 - val_loss: 0.8934 - val_acc: 0.6868
Epoch 9/10
47500/47500 [==============================] - 16s - loss: 0.9223 - acc: 0.6807 - val_loss: 0.8991 - val_acc: 0.6920
Epoch 10/10
47500/47500 [==============================] - 16s - loss: 0.8877 - acc: 0.6911 - val_loss: 0.8717 - val_acc: 0.6900